In [2]:
%run BackEnd_DataProcessing.ipynb
%run BackEnd_Plotters.ipynb
import ipywidgets as widgets
import pytimber
import numpy as np
import os
import pandas as pd
import pickle
import csv
db = pytimber.LoggingDB()

Populating the interactive namespace from numpy and matplotlib


# Data Collector
Assembles all useful Vacuum Gauge data for Unsupervised Learning <br>
Applies Filtering to ensure the data is of sufficient quality:
* Min Duration (6 hours)
* Min Data Points
* Fixed-Size Interpolant
* Must have Energy Readings

All data is stored in a compressed format on SWAN. <br>
Binary files such as these are NOT commited to the git repository.

In [4]:
max_fill = db.getLHCFillData(None)['fillNumber']

min_fill = 1800
fill_field = widgets.BoundedIntText(
    value=min_fill,
    min=min_fill,
    max=max_fill,
    step=1,
    description='Minimum FillNo:',
    disabled=False
)

display(fill_field)

max_fill = 2280

BoundedIntText(value=1800, description='Minimum FillNo:', max=7494, min=1800)

> Iterate over every fill in the selected range (above) <br>
    > For each fill find all availble probes <br>
        > For each probe apply filters <br>
            > If probe passes filter, process and store <br>
            > Add Probe to Catalogue File <br>
            

In [6]:
def duration_filter(t1,t2, verbose=False, hours = 5):
    date1 = datetime.datetime.fromtimestamp(t1)
    date2 = datetime.datetime.fromtimestamp(t2)
    duration = date2 - date1
    if duration < datetime.timedelta(hours=6):
        if verbose: print("Fill duration too short at {}".format(duration))
        return False
    else:
        return True

def energy_filter(beam_time, beam_energy, t1, t2, min_points = 100, delay=0.01, min_bins=10, verbose = False):
    duration = t2 - t1
    if np.isnan(beam_energy).any():
        if verbose: print("NAN in Beam Energy")
        return False
    elif len(beam_energy) < min_points:
        if verbose: print("Too few points in Beam Energy")
        return False
    elif np.min(beam_time) > t1+round(duration*delay) or np.max(beam_time) < t2-round(duration*delay):
        if verbose: print("Incomplete Beam Energy Coverage")
        return False
    elif np.any(np.histogram(beam_time,bins=min_bins,range=(t1,t2))[0] == 0):
        if verbose: print("No beam energy data for 1/{}th of data".format(min_bins))
        return False
    else:
        return True

def pressure_filter(time_readings, pressure_readings, t1, t2, min_points = 100, min_bins=10, verbose = False):
    if np.isnan(pressure_readings).any():
        if verbose: print("NAN in Pressure Readings")
        return False
    elif np.any(np.histogram(time_readings,bins=min_bins,range=(t1,t2))[0] == 0):
        if verbose: print("No Pressure data for 1/{}th of data".format(min_bins))
        return False
    elif len(pressure_readings) < min_points:
        if verbose: print("Too few points in Pressure Readings")
        return False
    else:
        return True
 
    
count, limit = 0, 10
for fillNo in range(fill_field.value,max_fill):
    print(">>>> Fill Number {} <<<<".format(fillNo))
    fill = db.getLHCFillData(fillNo)
    if fill is None: continue
    t1,t2   = fill['startTime'],fill['endTime']
    if duration_filter(t1,t2, verbose=True)==False: continue
    
    energy_variable = 'LHC.BOFSU:OFSU_ENERGY'
    beamEnergy=db.get(energy_variable, t1, t2)
    beam_time, beam_energy = beamEnergy[energy_variable]
    if energy_filter(beam_time, beam_energy, t1, t2, verbose=True)==False: continue
    
    folder = os.path.join(os.getcwd(),"data","probes",str(fillNo))
    gauges = db.get("VGPB.%.B.PR", t1, t2)
    for gauge_id, values in gauges.items():
        print("Examining '{}'".format(gauge_id))
        time_readings,pressure_readings = values[0],values[1]
        if pressure_filter(time_readings, pressure_readings, t1, t2, verbose = True)==False: continue
            
        file_name = "probe_%s_fill%d.p"%(gauge_id,fillNo)
        if os.path.isfile(os.path.join(folder,file_name)):
            print("{} already exists".format(file_name))
            continue
        else:
            pgd = processed_gauge_data(gauge_id, fillNo, 
                                       raw_beam_time = beam_time, raw_beam_energy =beam_energy,
                                       raw_time_readings=time_readings, raw_pressure_readings=pressure_readings)
            try: 
                pgd.generate_data()
            except ValueError:
                continue
                
            if not os.path.exists(folder):
                os.makedirs(folder)
            with open(os.path.join(folder,file_name), 'rb') as f:
                pickle.dump( pgd, open( os.path.join(folder,file_name), "wb" ))
                fields=['first','second','third']
                with open(r'UnsupervisedProbeCatalogue.csv','a') as f:
                    writer = csv.writer(f)
                    writer.writerow([gauge_id,"UNDEFINED","UNDETERMINED", gauge_id.split('.')[0],"Auto-generated", fillNo,])
                count += 1
                if count % limit == 0:
                    print("Saved {} probes to File".format(count))
        
#print(gauges)

>>>> Fill Number 2268 <<<<
Fill duration too short at 3:21:18.905000
>>>> Fill Number 2269 <<<<
Examining 'VGPB.1024.4R6.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.1032.4R6.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.1173.5L4.B.PR'
probe_VGPB.1173.5L4.B.PR_fill2269.p already exists
Examining 'VGPB.1173.5R4.B.PR'
probe_VGPB.1173.5R4.B.PR_fill2269.p already exists
Examining 'VGPB.1175.5L4.B.PR'
probe_VGPB.1175.5L4.B.PR_fill2269.p already exists
Examining 'VGPB.1175.5R4.B.PR'
probe_VGPB.1175.5R4.B.PR_fill2269.p already exists
Examining 'VGPB.12.5L6.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.129.5L8.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.1360.4R6.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.137.5L2.B.PR'
probe_VGPB.137.5L2.B.PR_fill2269.p already exists
Examining 'VGPB.138.5L2.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.139.5R8.B.PR'
probe_VGPB.139.5R8.B.PR_fill2269.p already exists
Examining 'VGPB.14

probe_VGPB.4.7R1.B.PR_fill2269.p already exists
Examining 'VGPB.4.7R2.B.PR'
probe_VGPB.4.7R2.B.PR_fill2269.p already exists
Examining 'VGPB.4.7R3.B.PR'
probe_VGPB.4.7R3.B.PR_fill2269.p already exists
Examining 'VGPB.4.7R4.B.PR'
probe_VGPB.4.7R4.B.PR_fill2269.p already exists
Examining 'VGPB.4.7R5.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.4.7R7.B.PR'
probe_VGPB.4.7R7.B.PR_fill2269.p already exists
Examining 'VGPB.4.7R8.B.PR'
probe_VGPB.4.7R8.B.PR_fill2269.p already exists
Examining 'VGPB.412.6R3.B.PR'
probe_VGPB.412.6R3.B.PR_fill2269.p already exists
Examining 'VGPB.413.4L7.B.PR'
probe_VGPB.413.4L7.B.PR_fill2269.p already exists
Examining 'VGPB.413.4R7.B.PR'
probe_VGPB.413.4R7.B.PR_fill2269.p already exists
Examining 'VGPB.414.6R3.B.PR'
probe_VGPB.414.6R3.B.PR_fill2269.p already exists
Examining 'VGPB.416.4L7.B.PR'
probe_VGPB.416.4L7.B.PR_fill2269.p already exists
Examining 'VGPB.416.4R7.B.PR'
probe_VGPB.416.4R7.B.PR_fill2269.p already exists
Examining 'VGPB.418.6L3.B.PR

probe_VGPB.907.5R6.B.PR_fill2269.p already exists
Examining 'VGPB.91.7L8.B.PR'
probe_VGPB.91.7L8.B.PR_fill2269.p already exists
Examining 'VGPB.910.5R6.B.PR'
probe_VGPB.910.5R6.B.PR_fill2269.p already exists
Examining 'VGPB.92.5L8.B.PR'
No Pressure data for 1/10th of data
Examining 'VGPB.935.4L1.B.PR'
probe_VGPB.935.4L1.B.PR_fill2269.p already exists
Examining 'VGPB.935.4L5.B.PR'
probe_VGPB.935.4L5.B.PR_fill2269.p already exists
Examining 'VGPB.935.4R1.B.PR'
probe_VGPB.935.4R1.B.PR_fill2269.p already exists
Examining 'VGPB.935.4R5.B.PR'
probe_VGPB.935.4R5.B.PR_fill2269.p already exists
Examining 'VGPB.938.4L1.B.PR'
probe_VGPB.938.4L1.B.PR_fill2269.p already exists
Examining 'VGPB.938.4L5.B.PR'
probe_VGPB.938.4L5.B.PR_fill2269.p already exists
Examining 'VGPB.938.4R1.B.PR'
probe_VGPB.938.4R1.B.PR_fill2269.p already exists
Examining 'VGPB.938.4R5.B.PR'
probe_VGPB.938.4R5.B.PR_fill2269.p already exists
Examining 'VGPB.98.5L2.B.PR'
probe_VGPB.98.5L2.B.PR_fill2269.p already exists
Examining

In [10]:
print(gauges['VGPB.1173.5L4.B.PR'])

(array([1.30604158e+09, 1.30604165e+09, 1.30604171e+09, 1.30604179e+09,
       1.30604187e+09, 1.30604196e+09, 1.30604207e+09, 1.30604221e+09,
       1.30604234e+09, 1.30604251e+09, 1.30604269e+09, 1.30604289e+09,
       1.30604295e+09, 1.30604308e+09, 1.30604345e+09, 1.30604353e+09,
       1.30604390e+09, 1.30604398e+09, 1.30604428e+09, 1.30604434e+09,
       1.30604440e+09, 1.30604473e+09, 1.30604534e+09, 1.30604541e+09,
       1.30604547e+09, 1.30604603e+09, 1.30604609e+09, 1.30604615e+09,
       1.30604682e+09, 1.30604689e+09, 1.30604696e+09, 1.30604791e+09,
       1.30604797e+09, 1.30604907e+09, 1.30604914e+09, 1.30604920e+09,
       1.30604926e+09, 1.30604932e+09, 1.30605004e+09, 1.30605011e+09,
       1.30605052e+09, 1.30605062e+09, 1.30605069e+09, 1.30605076e+09,
       1.30605082e+09, 1.30605088e+09, 1.30605094e+09, 1.30605100e+09,
       1.30605106e+09, 1.30605117e+09, 1.30605252e+09, 1.30605252e+09,
       1.30605258e+09, 1.30605259e+09, 1.30605267e+09, 1.30605274e+09,
     